### 三個步驟將原始的CSV輸出成加上鄰里欄位的CSV
### 一、輸入資料
### 二、將鄰里寫入DataFrame
### 三、輸出新檔案
##### 基於03，嘗試新調整

In [13]:
# 等待時間參數設定
# 強制等待
sleepTime = 3

# 顯性等待
waitTime = 3

# 每幾筆強制等待一次
timeToTakeABreak = 17

In [14]:
import re

# 數字轉中文字
def transZH(digit):
    return{'1':'一',
          '2':'二',
          '3':'三',
          '4':'四',
          '5':'五',
          '6':'六',
          '7':'七',
          '8':'八',
          '9':'九',
          '10':'十'}.get(digit, None)

def transDigit(zh):
    return{'一':'1',
          '二':'2',
          '三':'3',
          '四':'4',
          '五':'5',
          '六':'6',
          '七':'7',
          '八':'8',
          '九':'9',
          '十':'10'}.get(zh, None)

In [15]:
import re

# 將地址轉換成要輸入網頁查詢的條件
def getAddr(doorplate):
    # 正規
    # reg = r'(?P<city>[\u4E00-\u9FFF]{2}(?=[\u7e23|\u5e02])[\u7e23|\u5e02])?(?P<dist>[\u4E00-\u5339\u5341-\u9FFF]+(?=[\u9109|\u93ae|\u5e02|\u5340])[\u9109|\u93ae|\u5e02|\u5340])?(?P<road>[\u4E00-\u9FFF]+(?!\d)(?=[u8857|\u8def|\u5927\u9053])[u8857|\u8def|\u5927\u9053])(?P<sec>\d\u6bb5)?(?P<secZH>[\u4e00|\u4e8c|\u4e09|\u56db|\u4e94|\u516d|\u4e03|\u516b|\u4e5d|\u5341]\u6bb5)?(?P<ln>\d+\u5df7)?(?P<lnZH>[\u4E00-\u9FFF]{2}\u5df7)?(?P<aly>\d+\u5f04)?(?P<numberAllKind>((?P<number>\d+)\u865f(?P<zhi>\u4e4b\d+)?(?P<zhiZH>\u4e4b[\u4e00|\u4e8c|\u4e09|\u56db|\u4e94|\u516d|\u4e03|\u516b|\u4e5d|\u5341])?)|((?P<number2>\d+)\-(?P<zhi2>\d+)\u865f)|((?P<number3>\d+)\u4e4b(?P<zhi3>\d+)?(?P<zhi3ZH>[\u4e00|\u4e8c|\u4e09|\u56db|\u4e94|\u516d|\u4e03|\u516b|\u4e5d|\u5341])?\u865f))'
    reg = r'(?P<city>[\u4E00-\u9FFF]{2}(?=[\u7e23|\u5e02])[\u7e23|\u5e02])?(?P<dist>[\u4E00-\u5339\u5341-\u9FFF]+(?=[\u9109|\u93ae|\u5e02|\u5340])[\u9109|\u93ae|\u5e02|\u5340])?(?P<li>[\u4E00-\u9FFF]{2}(?=[\u6751|\u91cc])[\u6751|\u91cc])?(?P<lin>(?P<linD>\d+)(?=\u9130)\u9130)?(?P<road>[\u4E00-\u9FFF]+(?!\d)(?=[\u8857|\u8def|\u5927\u9053])[\u8857|\u8def|\u5927\u9053])(?P<sec>(?P<secD>\d)\u6bb5)?(?P<secZH>(?P<secZHD>[\u4e00|\u4e8c|\u4e09|\u56db|\u4e94|\u516d|\u4e03|\u516b|\u4e5d|\u5341])\u6bb5)?(?P<ln>(?P<lnD>\d+)\u5df7)?(?P<lnZH>[\u4E00-\u9FFF]{2}\u5df7)?(?P<aly>(?P<alyD>\d+)\u5f04)?(?P<numberAllKind>((?P<number>\d+)\u865f(?P<zhi>\u4e4b(?P<zhiD>\d+))?(?P<zhiZH>\u4e4b(?P<zhiZHD>[\u4e00|\u4e8c|\u4e09|\u56db|\u4e94|\u516d|\u4e03|\u516b|\u4e5d|\u5341]))?)|((?P<number2>\d+)\-(?P<zhi2D>\d+)\u865f)|((?P<number3>\d+)\u4e4b(?P<zhi3D>\d+)?(?P<zhi3ZHD>[\u4e00|\u4e8c|\u4e09|\u56db|\u4e94|\u516d|\u4e03|\u516b|\u4e5d|\u5341])?\u865f)|((?P<number4>\d+)\~\d+\u865f))?'
    address = re.match(reg, doorplate)
    
    try:
        # 縣市
        city = address['city']

        # 鄉鎮市區
        dist = address['dist']

        # 街路大道
        road = address['road']

        # 段
        if address['secD']!=None:
            sec = address['secD']
            sec = transZH(sec)
        else:
            sec = address['secZHD']


        # 巷
        if address['lnD']!=None:
            ln = address['lnD']
        else:
            ln = address['lnZH']

        # 弄
        aly = address['alyD']

        # 號
        if address['number']!=None:
            number = address['number']
        elif address['number2']!=None:
            number = address['number2']
        elif address['number3']!=None:
            number = address['number3']
        else:
            number = address['number4']

        # 之
        if address['zhiD']!=None:
            zhi = address['zhiD']
        elif address['zhiZHD']!=None:
            zhi = address['zhiZHD']
            zhi = transDigit(zhi)
        elif address['zhi2D']!=None:
            zhi = address['zhi2D']
        elif address['zhi3D']!=None:
            zhi = address['zhi3D']
        else:
            zhi = address['zhi3ZHD']
            zhi = transDigit(zhi)
        
        # 將號與之整合，以符合網頁查詢條件的規則
        if zhi!=None:
            number=''.join([number, '之', zhi])
        
        # 取得查詢條件  
        ls = [road, sec, ln, aly, number]
        # print(ls)
        
        # 檢查正規表達法 
        lsCheck = ['city', 'dist', 'road', 'sec', 'secZH', 'ln', 'lnZH', 'aly', 'numberAllKind', 'number', 'number2', 'number3', 'number4', 'zhiD', 'zhiZHD', 'zhi2D', 'zhi3D', 'zhi3ZHD']
        '''
        for check in lsCheck:
            print(check,'\t:', address[check])
        '''
        
        # 回傳查詢條件
        print('該列資料正常。')
        print(ls)
        return(ls)
    
    # 該列資料有問題，直接回傳空值串列
    except TypeError:
        ls = [None, None, None, None, None]
        print('該列資料有問題，回傳空值。')
        print(ls)
        return(ls)

In [16]:
import requests, random

from selenium import webdriver

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By

from time import sleep

from selenium.webdriver.chrome.options import Options

def startChrome():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    
    # Winsdows
    dr = webdriver.Chrome('./chromedriver', options=options) 

    # Mac OS
    # dr = webdriver.Chrome('./chromedriver', options=options) 

# 取得鄰里
def getLinAndLi(ls):
    # 設定查詢條件
    road = ls[0]
    sec = ls[1]
    ln = ls[2]
    aly = ls[3]
    number = ls[4]
    print("getLinAndLi()已接收引數如下：")
    print(ls)
    
    if ls == [None, None, None, None, None]:
        linAndLi = [None, None]
        print("該列資料有問題，不查詢，回傳空值。")
        print(linAndLi)
        return(linAndLi)
    
    # 設定強制等待
    time = random.random() * sleepTime
    sl = sleep(time)
   
    '''
    # 設定ChromeDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--start-maximized")
    options.add_argument("--incognito")
    options.add_argument("--disable-popup-blocking ")
    
    # 使selenium進入狀況
    dr = webdriver.Chrome('./chromedriver.exe')
    dr.maximize_window()
    dr.get('https://houseno.civil.taipei/') 
    sl
    '''
    
    # 無頭Chrome
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    
    dr = webdriver.Chrome('./chromedriver.exe', options=options)
    dr.maximize_window()
    dr.get('https://houseno.civil.taipei/') 
    
    # 強制等待1
    sl
    
    
    # 點擊「門牌檢索」
    dr.find_element(By.CSS_SELECTOR,"#collapsible_control_3 > img").click()
    

    
    # 切換iframe
    dr.switch_to.frame("Addrhis")
    dr.switch_to.frame("frm_new1")

    from selenium.common.exceptions import TimeoutException

    from selenium.webdriver.support.ui import WebDriverWait 

    from selenium.webdriver.support import expected_conditions as EC

    '''
    # 顯性等待
    try:
        # 滿足條件(時間內找到元素)，就往下一步
        WebDriverWait(dr, waitTime).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="A"]/input[1]')))
        print("已經可以輸入查詢條件。")

    except TimeoutException:
        dr.quit()
        linAndLi = [None, None]
        print("等待逾時，回傳空值。")
        print(linAndLi)
        return(linAndLi)
    '''
    
    # 輸入街路
    roadInput = dr.find_element(By.CSS_SELECTOR,'#A > input.txt')
    roadInput.send_keys(road)

    from selenium.webdriver.support.ui import Select

    # 選擇段
    if sec!=None:
        secSelect = Select(dr.find_element(By.CSS_SELECTOR,'#AA > select'))
        secOption = secSelect.select_by_value(sec)

    # 輸入巷
    if ln!=None:
        lnInput = dr.find_element(By.CSS_SELECTOR,'#AA > input:nth-child(2)')
        lnInput.send_keys(ln)

    # 輸入弄
    if aly!=None:
        alyInput = dr.find_element(By.CSS_SELECTOR,'#AA > input:nth-child(3)')
        alyInput.send_keys(aly)

    # 輸入號
    if number!=None:
        numberInput = dr.find_element(By.CSS_SELECTOR,'#AAA > input')
        numberInput.send_keys(number)
    
    # 強制等待2
    sl

    try:
        # 當你指定selenium進到一個iframe，它並不會自己隨意亂跑，而是要再經指示才會移動
        dr.switch_to.window(dr.current_window_handle)

        # 切換iframe
        dr.switch_to.frame("Addrhis")
        dr.switch_to.frame("frm_new1")
        
        # 點擊「立即檢索」
        # dr.find_element(By.CSS_SELECTOR,'#left_tb4 > tbody > tr > td > input[type=button]:nth-child(1)').click()
        searchNow = dr.find_element(By.XPATH,'//*[@id="left_tb4"]/tbody/tr/td/input[1]')
        # searchNow.click()
        dr.execute_script("arguments[0].click();", searchNow)

        
    except ElementClickInterceptedException:
        dr.quit()
        linAndLi = [None, None]
        print("查詢被攔截，回傳空值。")
        print(linAndLi)
        return(linAndLi)       
    
    except ElementNotInteractableException:
        dr.quit()
        linAndLi = [None, None]
        print("查詢不順利，回傳空值。")
        print(linAndLi)
        return(linAndLi)
    
    except NoSuchElementException:
        dr.quit()
        linAndLi = [None, None]
        print("查詢不順利，回傳空值。")
        print(linAndLi)
        return(linAndLi)    
    
    
    '''
    while alert_accept() == True:
        alert_accept()
    '''
    
    # 如果查詢條件找不到符合的資料，直接回傳空值串列
    try:
        WebDriverWait(dr, waitTime).until(EC.alert_is_present())
        alert = dr.switch_to.alert
        alert.accept()
        
        dr.quit()
        
        linAndLi = [None, None]
        print("查詢條件找不到符合的資料，回傳空值。")
        print(linAndLi)
        return(linAndLi)
        
    except TimeoutException:
        print("找到鄰里！")

    
    # 跳出iframe，回到原本的window
    # 一直搞不清楚selenium的運作原理：
    # selenium把element當作node，就是移動的依據
    # 當你指定selenium進到一個iframe，它並不會自己隨意亂跑，而是要再經指示才會移動
    dr.switch_to.window(dr.current_window_handle)

    # 切換iframe
    dr.switch_to.frame("Addrhis")
    dr.switch_to.frame("frm_new2")

    # 擷取里
    getLi = dr.find_elements_by_css_selector("#tb_no_border > tbody > tr > td:nth-child(1) > a:nth-child(2) > font:nth-child(2)")
    li = getLi[0].text

    # 擷取鄰
    getLin = dr.find_elements_by_css_selector("#tb_no_border > tbody > tr > td:nth-child(1) > a:nth-child(2) > font:nth-child(3)")
    lin =getLin[0].text
        
    '''
    # 是否可以在這裡更動輸入此函數的資料，重頭再做一次原本的工作?
    except UnexpectedAlertPresentException:
        dr.switch_to.alert.accept()
        print("查詢條件查無結果！")
    '''
    
    # 關閉ChromeDriver
    dr.quit()
    
    # 回傳鄰里
    linAndLi = [li, lin]
    print(linAndLi)
    return(linAndLi)

In [51]:
import numpy as np

import pandas as pd

def appendToDF(row, linAndLi, data):
    li = linAndLi[0]
    lin = linAndLi[1]
    newData = data
    
    newData.loc[row,'Li'] = li
    newData.loc[row,'Lin'] = lin
    
    print("資料已經寫入newData。")
    return(newData)

In [18]:
import os

def exportNewFile(newData, newFileName):
    strNewData = newData.to_csv(index=False)
    fp = open(f'{newFileName}.csv', 'w', encoding = 'utf-8')
    fp.write(strNewData)
    fp.close

# 一、輸入資料
#### 請輸入要讀取的資料...

In [40]:
# dataSrc = ""
dataSrc = "./taipeiGovernmentAgency01.csv"
# dataSrc = "./109年度臺北市公立幼兒園名錄.csv"
data = pd.read_csv(dataSrc)
data

,Name,Longtitude,Latitude,Address,Area,St,AgencyCode,TEL,TEL2
0,臺北市政府,121.563997,25.037531,臺北市信義區市府路1號,信義區,市府路,379000000A,02-27208889,NaN
1,臺北市政府秘書處,121.563997,25.037531,臺北市信義區市府路1號10、11樓,信義區,市府路,379010000A,02-27208889,NaN
2,臺北市市政大樓公共事務管理中心,121.563997,25.037531,臺北市信義區市府路1號12樓北區,信義區,市府路,379010300A,02-27258614#8614,NaN
3,臺北市政府民政局,121.564433,25.037542,臺北市信義區市府路1號9樓中央區,信義區,市府路,379020000A,02-27208889,02-27256180
4,臺北市孔廟管理委員會,121.516179,25.072762,臺北市大同區大龍街275號,大同區,大龍街,379020200E,02-25923934,NaN
...,...,...,...,...,...,...,...,...,...
386,臺北市政府勞動局,121.563997,25.037531,臺北市信義區市府路1號5樓北區,信義區,市府路,379760000J,02-27208889,NaN
387,臺北市就業服務處,121.501374,25.033444,臺北市萬華區艋舺大道101號8樓,萬華區,艋舺大道,379760100J,02-23085230,NaN
388,臺北市勞動力重建運用處,121.501374,25.033444,臺北市萬華區艋舺大道101號5樓,萬華區,艋舺大道,379760200J,02-23381600,NaN
389,臺北市勞動檢查處,121.501374,25.033444,臺北市萬華區艋舺大道101號7樓,萬華區,艋舺大道,379760300J,02-23086101,NaN


In [46]:
data.iloc[181,:]
data.iloc[181,3]='臺北市松山區民權東路５段1號'
data.iloc[181,3]

'臺北市松山區民權東路５段1號'

# 二、將鄰里寫入DataFrame
#### 請輸入門牌地址的欄位，以及想要加上鄰里的列...
#### 請確認你的chromedriver和這個.ipynb檔案是否在同一個資料夾！！！

In [47]:
# doorplateColumn = "門牌地址的欄位"
doorplateColumn = "Address"

# 全部的列
# rangeOfRows = range(0,len(data))
# 部分的列
# rangeOfRows = range(起始列,終點列（不含）)
rangeOfRows = range(181,len(data))

# 讀取檔案
# newData = data
# 爬取道一半的表
newData = data

doorplateList = newData[doorplateColumn]
doorplateList

0            臺北市信義區市府路1號
1      臺北市信義區市府路1號10、11樓
2       臺北市信義區市府路1號12樓北區
3       臺北市信義區市府路1號9樓中央區
4          臺北市大同區大龍街275號
             ...        
386      臺北市信義區市府路1號5樓北區
387     臺北市萬華區艋舺大道101號8樓
388     臺北市萬華區艋舺大道101號5樓
389     臺北市萬華區艋舺大道101號7樓
390        臺北市士林區士東路301號
Name: Address, Length: 391, dtype: object

In [48]:
for item in rangeOfRows:
    
    if item%timeToTakeABreak==0:
        # 強制等待3
        print("\n*** Time to take a break! ***\n")
        sleep(random.random() * sleepTime)
        print("********* WORK  OUT *********\n")
    
    print(f'=====第{item:4.0f}列資料=====')
    
    doorplate = doorplateList[item]
   
    conditions = getAddr(doorplate)

    linAndLi = getLinAndLi(conditions)
    
    newData = appendToDF(item, linAndLi, newData)
    
    print(f'=====================\n') 

=====第 181列資料=====
該列資料正常。
['民權東路', None, None, None, '1']
getLinAndLi()已接收引數如下：
['民權東路', None, None, None, '1']
找到鄰里！
['恆安里', '001鄰']
資料已經寫入newData。

=====第 182列資料=====
該列資料正常。
['舊莊街', None, None, None, None]
getLinAndLi()已接收引數如下：
['舊莊街', None, None, None, None]
找到鄰里！
['中研里', '010鄰']
資料已經寫入newData。

=====第 183列資料=====
該列資料正常。
['星雲街', None, None, None, '121']
getLinAndLi()已接收引數如下：
['星雲街', None, None, None, '121']
找到鄰里！
['清白里', '030鄰']
資料已經寫入newData。

=====第 184列資料=====
該列資料正常。
['天玉街', None, None, None, '12']
getLinAndLi()已接收引數如下：
['天玉街', None, None, None, '12']
找到鄰里！
['天玉里', '003鄰']
資料已經寫入newData。

=====第 185列資料=====
該列資料正常。
['文林路', None, '615', None, '20']
getLinAndLi()已接收引數如下：
['文林路', None, '615', None, '20']
找到鄰里！
['德華里', '010鄰']
資料已經寫入newData。

=====第 186列資料=====
該列資料正常。
['興南街', None, None, None, '62']
getLinAndLi()已接收引數如下：
['興南街', None, None, None, '62']
找到鄰里！
['東新里', '007鄰']
資料已經寫入newData。


*** Time to take a break! ***

********* WORK  OUT *********

=====第 187列資料=====
該列資料正常。


# 三、輸出新檔案
#### 請輸入新檔名...

In [50]:
# newFileName = "新檔名"
newFileName = "taipeiGovernmentAgencyAddLinAndLi181_391.csv"

output = exportNewFile(newData, newFileName)

In [49]:
# 檢查DataFrame是否有加上鄰里
newData[181:391]

,Name,Longtitude,Latitude,Address,Area,St,AgencyCode,TEL,TEL2,里,鄰
181,臺北市松山區三民國民小學,121.565899,25.063282,臺北市松山區民權東路５段1號,松山區,民權東路,379081400Y,02-27646080,NaN,恆安里,001鄰
182,臺北市南港區胡適國民小學,121.624044,25.037033,臺北市南港區舊莊街１段,南港區,舊莊街,379081600Y,02-27824949,02-27824949,中研里,010鄰
183,臺北市內湖區康寧國民小學,121.594931,25.080113,臺北市內湖區星雲街121號,內湖區,星雲街,379081700Y,02-27901237,NaN,清白里,030鄰
184,臺北市士林區天母國民小學,121.527533,25.119791,臺北市士林區天玉街12號,士林區,天玉街,379081900Y,02-28723336,NaN,天玉里,003鄰
185,臺北市士林區文昌國民小學,121.521985,25.099347,臺北市士林區文林路615巷20號,士林區,文林路,379082000Y,02-28365411,NaN,德華里,010鄰
...,...,...,...,...,...,...,...,...,...,...,...
386,臺北市政府勞動局,121.563997,25.037531,臺北市信義區市府路1號5樓北區,信義區,市府路,379760000J,02-27208889,NaN,西村里,008鄰
387,臺北市就業服務處,121.501374,25.033444,臺北市萬華區艋舺大道101號8樓,萬華區,艋舺大道,379760100J,02-23085230,NaN,富福里,007鄰
388,臺北市勞動力重建運用處,121.501374,25.033444,臺北市萬華區艋舺大道101號5樓,萬華區,艋舺大道,379760200J,02-23381600,NaN,富福里,007鄰
389,臺北市勞動檢查處,121.501374,25.033444,臺北市萬華區艋舺大道101號7樓,萬華區,艋舺大道,379760300J,02-23086101,NaN,富福里,007鄰
